# 02_bbox_to_grid — Define a UTM target grid & write a template GeoTIFF

Goal:
- Read ROI (west/east/south/north) from `../config/project.yaml`.
- Compute an appropriate **UTM EPSG** for the ROI (or use `target_grid.crs` if provided).
- Build a **200 m** target grid that fully covers the ROI.
- Save an **empty template GeoTIFF** (`_grid_template.tif`) — all subsequent datasets will be reprojected to match this grid.
- Load `../data/outputs/discovered_assets.csv` (from `01_discover_links.ipynb`) to confirm discovered inputs.

This notebook does **not** reproject any data yet; it's only defining the target grid.


In [1]:
import os
from pathlib import Path
import math
import numpy as np
import pandas as pd
import yaml
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS
from affine import Affine
import pyproj
from shapely.geometry import box

print('✅ Imports ok')

✅ Imports ok


## 1) Load config and ROI

In [2]:
cfg_path = Path('../config/project.yaml')
cfg = {}
if cfg_path.exists():
    with open(cfg_path, 'r') as f:
        cfg = yaml.safe_load(f)
        print('Loaded config from', cfg_path)
else:
    print('⚠️ No config found; using defaults.')

roi = cfg.get('roi', {
    'west': -118.48,
    'east': -117.47,
    'south': 34.235,
    'north': 35.365,
})
print('ROI (EPSG:4326):', roi)

tgrid = cfg.get('target_grid', {})
resolution = float(tgrid.get('resolution', 200.0))  # meters
nodata = tgrid.get('nodata', -9999)
dtype = tgrid.get('dtype', 'float32')
template_path = tgrid.get('template_path', '../data/outputs/aligned/_grid_template.tif')
Path(template_path).parent.mkdir(parents=True, exist_ok=True)
print('Target pixel size (m):', resolution)
print('Template path:', template_path)

Loaded config from ../config/project.yaml
ROI (EPSG:4326): {'west': -118.48, 'east': -117.47, 'south': 34.235, 'north': 35.365}
Target pixel size (m): 200.0
Template path: ../data/outputs/aligned/_grid_template.tif


## 2) Choose UTM zone (or use explicit CRS from config)
- If `target_grid.crs` is set in the config, we use it.
- Otherwise, compute UTM EPSG from the ROI centroid.

UTM EPSG rules:
- Zone = floor((lon + 180)/6) + 1
- EPSG: 326## for northern hemisphere, 327## for southern hemisphere.

In [3]:
def utm_epsg_from_lonlat(lon, lat):
    zone = int(math.floor((lon + 180) / 6) + 1)
    north = lat >= 0
    epsg = 32600 + zone if north else 32700 + zone
    return epsg, zone, north

explicit_crs = tgrid.get('crs')
if explicit_crs:
    target_epsg = int(str(explicit_crs).split(':')[-1]) if 'EPSG' in str(explicit_crs).upper() else explicit_crs
    print('Using explicit target CRS from config:', explicit_crs)
else:
    lon_c = 0.5 * (roi['west'] + roi['east'])
    lat_c = 0.5 * (roi['south'] + roi['north'])
    target_epsg, zone, north = utm_epsg_from_lonlat(lon_c, lat_c)
    print(f'Computed UTM: EPSG:{target_epsg} (zone {zone}, {"north" if north else "south"}) from centroid ({lon_c:.3f},{lat_c:.3f})')

crs = CRS.from_epsg(int(target_epsg)) if not isinstance(target_epsg, CRS) else target_epsg
print('Target CRS:', crs)

Computed UTM: EPSG:32611 (zone 11, north) from centroid (-117.975,34.800)
Target CRS: EPSG:32611


## 3) Project ROI to target CRS and compute grid geometry
We project the geographic ROI (EPSG:4326) corners into the UTM CRS, then create an **upper-left aligned** grid
with `resolution` meters. We **snap** the grid to whole pixels using floor/ceil so the ROI is fully covered.

In [4]:
proj = pyproj.Transformer.from_crs('EPSG:4326', crs, always_xy=True)
xw, ys = proj.transform(roi['west'], roi['south'])
xe, yn = proj.transform(roi['east'], roi['north'])

xmin, xmax = min(xw, xe), max(xw, xe)
ymin, ymax = min(ys, yn), max(ys, yn)

res = resolution
xmin_snap = math.floor(xmin / res) * res
ymax_snap = math.ceil(ymax / res) * res
xmax_snap = math.ceil(xmax / res) * res
ymin_snap = math.floor(ymin / res) * res

width = int(round((xmax_snap - xmin_snap) / res))
height = int(round((ymax_snap - ymin_snap) / res))
transform = from_origin(xmin_snap, ymax_snap, res, res)

print('Projected extent (m):')
print(' xmin, xmax:', xmin_snap, xmax_snap)
print(' ymin, ymax:', ymin_snap, ymax_snap)
print('Grid size (height, width):', height, width)
print('Affine transform:', transform)

Projected extent (m):
 xmin, xmax: 363600.0 457400.0
 ymin, ymax: 3789200.0 3913800.0
Grid size (height, width): 623 469
Affine transform: | 200.00, 0.00, 363600.00|
| 0.00,-200.00, 3913800.00|
| 0.00, 0.00, 1.00|


## 4) Write empty template GeoTIFF (COG-ready)

In [5]:
profile = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': dtype,
    'crs': crs,
    'transform': transform,
    'tiled': True,
    'compress': 'deflate',
    'nodata': nodata,
}
data = np.full((height, width), nodata, dtype=np.float32)
out_path = Path(template_path)
out_path.parent.mkdir(parents=True, exist_ok=True)
with rasterio.open(out_path, 'w', **profile) as dst:
    dst.write(data, 1)
print('✅ Wrote template grid to:', out_path)

✅ Wrote template grid to: ../data/outputs/aligned/_grid_template.tif


---
### Notes
- **SMAP & NISAR** granules are typically **HDF5** (`.h5`); they will be resampled later.
- **Clay fraction** is a **GeoTIFF** (`.tif`); reprojection is straightforward.
- **NDVI** here is a **binary** (`.int16` or similar); you will need its shape/transform/CRS from documentation or sidecar metadata when reprojecting.
- All resampling in later notebooks will **reproject to this template** using `rasterio.warp.reproject` or `rioxarray.reproject_match`.
